**Project**

**Project Topic:** Image Filteration And Noise Reduction using CNN

**Course:** Introduction To Machine Learning

**Group Members:**


1.   Shahzaib Hamza (FA20-BSSE-0062)
2.   Sarfaraz Illahi Soomro (FA20-BSSE-0027)
3.   Syed Maher Ali Shah (FA20-BSSE-0061)



**Section 1:**(Importing Necessary Libraries)

In [ ]:
!pip install tensorflow numpy matplotlib opencv-python ipywidgets

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import cv2
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.3 MB/s eta 0:00:00


**Section 2:** MNIST Denoising Model Data Preprocessing

This code snippet loads the MNIST dataset, a collection of handwritten digits, for training a denoising model. Firstly, it loads the dataset and then preprocesses it by scaling the pixel values to a range between 0 and 1. Finally, it expands the dimensions of the data to include a channel dimension, necessary for convolutional operations. This prepares the dataset for training a denoising model effectively.

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

11490434/11490434 [==============================] - 0s 0us/step


**Section 3:** (interactive Image Processing Toolbox with Denoising)

This script sets up an interactive environment for image processing, focusing on denoising. It begins by loading the necessary libraries and defining functions for image manipulation. The MNIST dataset is preprocessed to train a denoising model, which is then utilized in the interactive environment. Users can upload an image and apply various filters such as grayscale conversion, edge detection, sharpening, and more. Additionally, they can add salt-and-pepper noise to images and denoise them using the trained model. The interface is enhanced with interactive widgets for ease of use.

In [ ]:
denoising_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(1, (3, 3), activation='sigmoid', padding='same')
])
denoising_model.compile(optimizer='adam', loss='binary_crossentropy')

**Section 4:** (Denoising Model Training)

In this step, the denoising model is trained using the prepared noisy data. Gaussian noise with a specified factor is added to the MNIST dataset, ensuring variability in training samples. The noisy images are clipped to maintain pixel values within the valid range. The model is then trained for a specified number of epochs with a defined batch size. Training progress is monitored using validation data. This step aims to optimize the model for effective denoising performance.

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

denoising_model.fit(x_train_noisy, x_train, epochs=10, batch_size=128, shuffle=True, validation_data=(x_test_noisy, x_test))

Epoch 1/10
469/469 [==============================] - 122s 256ms/step - loss: 0.1697 - val_loss: 0.1169
Epoch 2/10
469/469 [==============================] - 115s 245ms/step - loss: 0.1136 - val_loss: 0.1087
Epoch 3/10
469/469 [==============================] - 116s 247ms/step - loss: 0.1075 - val_loss: 0.1054
Epoch 4/10
469/469 [==============================] - 114s 243ms/step - loss: 0.1046 - val_loss: 0.1030
Epoch 5/10
469/469 [==============================] - 114s 244ms/step - loss: 0.1028 - val_loss: 0.1012
Epoch 6/10
469/469 [==============================] - 117s 249ms/step - loss: 0.1015 - val_loss: 0.1001
Epoch 7/10
469/469 [==============================] - 115s 245ms/step - loss: 0.1004 - val_loss: 0.0993
Epoch 8/10
469/469 [==============================] - 115s 244ms/step - loss: 0.0997 - val_loss: 0.0985
Epoch 9/10
469/469 [==============================] - 112s 240ms/step - loss: 0.0990 - val_loss: 0.0985
Epoch 10/10
469/469 [==============================] - 117s 250m

**Section 5:** Saving The Denoising Model

In [ ]:
denoising_model.save('denoising_cnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Section 6:** Defining Image Processing Helper Functions

This section defines helper functions essential for image processing tasks within the interactive environment. The upload_and_preprocess_image function facilitates the upload and preprocessing of images, converting them to the required format for processing and denoising. The apply_filter function applies a specified model's filter to an input image, returning the filtered result. These functions streamline the image processing workflow, enabling efficient manipulation and denoising of uploaded images.

In [ ]:
def upload_and_preprocess_image():
    uploaded = files.upload()
    for fn in uploaded.keys():
        image = cv2.imread(fn)
        colored_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.resize(gray_image, (28, 28))
        gray_image = gray_image.astype('float32') / 255.0
        gray_image = np.expand_dims(gray_image, axis=-1)
        gray_image = np.expand_dims(gray_image, axis=0)
        return colored_image, gray_image, fn

def apply_filter(model, image):
    return model.predict(image)

**Section 7:** Image Display And Analysis Function

This function, display_image, is responsible for displaying the original and filtered images side by side, along with their corresponding histograms. It creates a figure with three subplots: the first subplot displays the original image, the second displays the filtered image with a specified title, and the third shows the histogram of pixel intensity values in the filtered image. This function aids in visualizing the effects of various filters and provides insights into the distribution of pixel values within the processed image.

In [ ]:
def display_image(original, filtered, title):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(original)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(filtered, cmap='gray')
    plt.title(title)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.hist(filtered.flatten(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram')
    plt.show()

**Section 8:** Image Processing Filters and Manipulations

This set of functions encompasses various image processing filters and manipulations for enhancing or modifying images. convert_to_grayscale converts a colored image to grayscale. apply_edge_detection applies an edge detection filter to detect edges in the image. apply_sharpening sharpens the image by applying a specific kernel. add_salt_and_pepper_noise adds salt-and-pepper noise to the image to simulate noisy conditions. apply_denoising utilizes the denoising model to remove noise from the image. mean_pass_filter applies a mean pass filter to the image to blur it slightly. convert_green_to_red converts green regions in the image to red, altering the color composition. These functions offer a range of options for image manipulation and enhancement within the interactive environment.

In [ ]:
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def apply_edge_detection(image):
    kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    edge_detected = cv2.filter2D(image, -1, kernel)
    return edge_detected

def apply_sharpening(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

def add_salt_and_pepper_noise(image, amount=0.04, salt_vs_pepper=0.5):
    noisy_image = image.copy()
    num_salt = np.ceil(amount * image.size * salt_vs_pepper)
    num_pepper = np.ceil(amount * image.size * (1.0 - salt_vs_pepper))

    # Add Salt noise
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    noisy_image[tuple(coords)] = 1

    # Add Pepper noise
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    noisy_image[tuple(coords)] = 0

    return noisy_image

def apply_denoising(image):
    return apply_filter(denoising_model, image).reshape(28, 28)

def mean_pass_filter(image):
    kernel = np.ones((3, 3), np.float32) / 9
    return cv2.filter2D(image, -1, kernel)

def convert_green_to_red(image):
    red_image = image.copy()
    lower_green = np.array([0, 100, 0])
    upper_green = np.array([100, 255, 100])
    mask = cv2.inRange(red_image, lower_green, upper_green)
    red_image[mask > 0] = [255, 0, 0]
    return red_image

**Section 9:** Interactive Image Processing Widgets

This step introduces interactive widgets to facilitate real-time image processing and manipulation. After uploading and preprocessing an image, users can interact with various buttons to apply different filters and transformations. The provided buttons include options for converting the image to grayscale, performing edge detection, sharpening the image, adding salt-and-pepper noise, denoising the image using the trained model, applying a mean pass filter, and converting green regions to red. Each button click triggers a specific image processing function, and the results are displayed instantly for user inspection. This interactive interface empowers users to explore different image processing techniques effortlessly.

In [ ]:
colored_image, gray_image, filename = upload_and_preprocess_image()

plt.imshow(colored_image)
plt.title('Original Image')
plt.axis('off')
plt.show()

buttons = {
    'Grayscale': widgets.Button(description='Convert to Grayscale'),
    'Edge Detection': widgets.Button(description='Edge Detection'),
    'Sharpening': widgets.Button(description='Sharpening'),
    'Add Noise': widgets.Button(description='Add Salt and Pepper Noise'),
    'Denoising': widgets.Button(description='Denoising'),
    'Mean Pass Filter': widgets.Button(description='Mean Pass Filter'),
    'Green to Red': widgets.Button(description='Convert Green to Red')
}

def on_grayscale_clicked(b):
    grayscale_image = convert_to_grayscale(colored_image)
    display_image(colored_image, grayscale_image, 'Grayscale')

def on_edge_detection_clicked(b):
    edge_detected_image = apply_edge_detection(cv2.cvtColor(colored_image, cv2.COLOR_RGB2GRAY))
    display_image(colored_image, edge_detected_image, 'Edge Detection')

def on_sharpening_clicked(b):
    sharpened_image = apply_sharpening(cv2.cvtColor(colored_image, cv2.COLOR_RGB2GRAY))
    display_image(colored_image, sharpened_image, 'Sharpened')

def on_add_noise_clicked(b):
    noisy_image = add_salt_and_pepper_noise(colored_image)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(colored_image)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(noisy_image)
    plt.title('Noisy Image')
    plt.axis('off')
    plt.show()

def on_denoising_clicked(b):
    noisy_image = add_salt_and_pepper_noise(colored_image)
    denoised_image = apply_denoising(gray_image)
    display_image(noisy_image, colored_image, 'Denoised')

def on_mean_pass_filter_clicked(b):
    mean_filtered_image = mean_pass_filter(convert_to_grayscale(colored_image))
    display_image(colored_image, mean_filtered_image, 'Mean Pass Filter')

def on_green_to_red_clicked(b):
    red_image = convert_green_to_red(colored_image)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(colored_image)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(red_image)
    plt.title('Green to Red')
    plt.axis('off')
    plt.show()

buttons['Grayscale'].on_click(on_grayscale_clicked)
buttons['Edge Detection'].on_click(on_edge_detection_clicked)
buttons['Sharpening'].on_click(on_sharpening_clicked)
buttons['Add Noise'].on_click(on_add_noise_clicked)
buttons['Denoising'].on_click(on_denoising_clicked)
buttons['Mean Pass Filter'].on_click(on_mean_pass_filter_clicked)
buttons['Green to Red'].on_click(on_green_to_red_clicked)

for button in buttons.values():
    display(button)

NameError: name 'upload_and_preprocess_image' is not defined